In [1]:
import sys
# %store -r SYSPATHS
SYSPATHS = {"tool":"../../../tool"}
sys.path.append(SYSPATHS['tool'])
import Khfeature as kf
import Khcore1 as kc1
from imp import reload
import MyEcharts as me
import pandas as pd
import numpy as np
import pylab as pl
import seaborn as sns
import os
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn import linear_model
import pickle

reload(kc1)
reload(kf)

<module 'Khfeature' from '../../../tool\\Khfeature.py'>

In [357]:
step = "1min"  ## 计算步进间隔
## 选用三个 不同的时间尺度
temp_window = "15min"  ## 平均温度计算窗口: 平均温度的窗口尽量小, 体现其短时作用, 可以快速结束报警, 尺度也不能太短, 防止label毛刺 
roll_window = '2H'  ## 滑动计算窗口: 滑动窗口用于计算Q1,Q3,IQR 和 diff, 滑动窗口下的统计量, 时间可以稍微长点
slope_window = "3H"  ## 斜率计算窗口: 滑动温度斜率窗口, 因为温度存在小范围周期波动, 所以需要将时间窗口放大
th = 61.5  ## 报警温度阈值
rate = 0.55  ## 报警概率阈值

## 一、获取原始数据

In [358]:
### 1. 获取原始数据
names = ["TE_1003a","TE_1003b","TE_1003c"]
datas=  []
for name in names:
    datas.append(kc1.ReadRawData(name))
data = pd.concat(datas,axis=1)
data.columns = names
data['PT_4001'] = kc1.ReadRawData("PT_4001",month="2019_06",factory="中碳能源")
data['TE_1003'] = kc1.ReadRawData("TE_1003",column='value', factory='中碳能源', month='2019_06')
data= data[data.index>"2019-06-13 19:00:00"]   ## 2019-06-13 19:00:00 之前属于数据测点调试阶段
data = data.resample(step).mean()

## 二、自动打标签
    根据浓缩段平均温度 生成label

In [370]:
s2 = kf.SingleTsFeature(data['TE_1003'])
s2.add_label(th=th,left_window = '60min',right_window="1min",merge_window="20min",filename="label_auto")
label = s2.GetDf()[['label']]

save image as label_auto
add label successful 


../../../tool/Khfeature.py:118: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  t['diff']= ((pd.Series(t.index).diff().dt.total_seconds()).values/S)
../../../tool/Khfeature.py:119: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  t['diff']= t['diff'].fillna(100)


##### 查看自动标签效果

In [360]:
temp = pd.concat([data,label],axis=1)
temp['label'] = temp['label']*70
temp['PT_4001'] = temp['PT_4001']*100
me.Plot_LineBar(temp,name='Temp_raw_label_06',overwrite=True)

## 三、特征加工

### 1. 浓缩温度 temp : temp_window 下的 平均温度最大值
### 2. 浓缩温度 diff ： roll_window 下的 温度差值均值
### 3. 浓缩温度 iqr ：roll_window 下的 平均温度IQR
### 4. 浓缩温度 slope ：slope_window 下的 拟合斜率

##### 利用 SingleTsFeature 对 TE_1003 加工特征变量

In [361]:
### 1. 加工特征
s1 = kf.SingleTsFeature(data["TE_1003"]-th)  ##  将原始温度 减去 温度阈值 加工特征
s1.add_roll_quantile(q = 0.75,window=roll_window,varname = "q3")
s1.add_roll_quantile(q = 0.25,window=roll_window,varname = "q1")
s1.add_slope(window=slope_window)

add q3 successful 
add q1 successful 


../../../tool/Khfeature.py:210: FutureWarning: Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True' to keep the current behaviour, and you can pass 'raw=False' to silence this warning
  self._df[varname] = self._df[self._column].rolling(window=window).apply(kc1.gradient)


add_slope successful 


In [369]:
###  2. 某些特征 用户可自行生成
df = s1.GetDf()
df['iqr'] = df1['q3'] -df1['q1']
df['diff'] = data[['TE_1003a','TE_1003b','TE_1003c']].max(axis=1)-data['TE_1003']  ## 加工 温度差值 diff
df['diff'] = df['diff'].rolling(roll_window).mean()
df['temp'] = df['TE_1003'].rolling(temp_window).max()
df1 = pd.concat([df,label],axis=1)
me.Plot_LineBar(df1,columns = ["temp",'diff','iqr','slope','label'],name='Feature_label_06',overwrite=True)

In [5]:
df1 = pd.read_csv("./data/process/Temp06.csv",index_col=0)

In [1]:
import matplotlib.pyplot as plt 
columns = ["temp",'diff','iqr','slope']
ylims = [(-7,6),(0,3.5),(-0.5,2),(-0.03,0.03)]
pl.figure(figsize=(20,12))
for index,col in enumerate(columns):
    plt.rcParams['font.family'] = 'SimHei' 
    plt.rcParams['font.size'] = 20
    plt.rcParams['axes.unicode_minus'] = False
    temp1 = df1.copy()
    pl.subplot(2,2,index+1)
    temp1['label'] =temp1['label'].map({0:u'正常',1:u'异常'})
    temp1['index'] = temp1.index
    temp1 = temp1.pivot_table(index = 'index',columns='label',values=col)
    temp1 = temp1[['正常','异常']]
    sns.set_style('whitegrid')
    sns.boxplot(data = temp1)
    plt.title(col)
    plt.ylim(*ylims[index])
    plt.xlabel('')
pl.show()

## 四、模型训练

### 1. 提取模型X,Y 

In [403]:
# scaler = MinMaxScaler()
# columns = ["temp",'diff','iqr','slope']
# df2 = df1.copy()
# X= df2[columns].values
# X1=scaler.fit_transform(X)
# Y = df2[['label']].values

# ## 保存因变量模型的 scale
# pickle.dump(scaler, open('./model/scaler.pkl', 'wb'))

In [404]:
# scaler.data_min_,scaler.data_max_,scaler.data_range_

### 2. 采用 Logistic Regression 模型训练

In [409]:
columns = ["temp",'diff','iqr','slope']
df2 = df1.copy()
X1 = df2[columns].values
Y = df2[['label']].values
model = linear_model.LogisticRegression(penalty = 'none',solver = 'saga', class_weight = 'balanced', max_iter=1000)
model.fit(X1,Y)
print(model.score(X1,Y))
print (model.coef_,model.intercept_)
df2['Logistic'] = model.predict_proba(X1)[:,1]

/root/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.9256658595641647
[[ 2.8438591   0.75461285 -0.57411258 -1.05654281]] [1.50807295]


In [410]:
## 保存因变量模型的 model
pickle.dump(model, open('./model/model.pkl', 'wb'))

### 3. 阈值规则修正label

In [411]:
## 将温度超过 th 的 label 直接置成报警概率
temp = df2.loc[df2[df2['TE_1003']>0].index,"Logistic"]
df2.loc[temp[temp<rate].index,"Logistic"] = rate
df2['rate'] = rate

### 4. 模型效果

In [415]:
me.Plot_LineBar(df2[df2.index<"2019-06-20 12:00:00"],columns = ['TE_1003','Logistic','label','rate'],\
                name="Temp_Label_Logistic_0",overwrite=True)

In [417]:
me.Plot_LineBar(df2[df2.index>="2019-06-20 12:00:00"],columns = ['TE_1003','Logistic','label','rate'],\
                name="Temp_Label_Logistic_1",overwrite=True)